In [3]:
from neo4j import GraphDatabase

In [4]:
class BlockChainDB(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!

    def close(self):
        self._driver.close()
        
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    
    @staticmethod
    def _new_output(tx, amount, script_pubkey,tx_id,out_index):
        result = tx.run("MATCH (n:transaction {id : $tx_id}) "
                        "CREATE p = (:output {out_index:$out_index, script_pubkey=$script_pubkey})<-[:CREATES]-(n) "
                        "RETURN p ", tx_id = tx_id, script_pubkey=script_pubkey, out_index=out_index, local_index=local_index)
        return result.single()
    
    
    @staticmethod
    def _new_tx(version, locktime, tx_id, inputs, outputs, segwit):
        result = tx.run("CREATE (n:transaction {id:$tx_id, version:$version, segwit:$segwit, locktime=$locktime}) "
                        "RETURN n",
                        tx_id=tx_id, segwit=segwit, version=version, locktime=locktime)
        
        for index,tx_out in enumerate(outputs):
            self._new_output(tx_out.amount, tx_out.script_pubkey, tx_id, index)
            
        for tx_in in inputs:
            result = tx.run("MATCH (t:transaction {id:$tx_id}), (o:output {index:$input_index})--(:transaction {id:$prev_tx}) "
                        "CREATE (t)-[:SPENT]->(o)",
                        tx_id=tx_id  input_index=tx_in.prev_index, prev_tx=tx.prev_tx)
        
        return result.single()
    
    
    @staticmethod
    def _new_block(block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        _height = tx.run( "MATCH (u:block) RETURN COUNT (u) ").single()[0] 
        result = tx.run("CREATE (n:block {id:$block_id, version:$version , prev_block:$prev_block, merkle_root:$merkle_root, timestamp:$timestamp, bits:$bits, nonce:$nonce, n_tx:n_tx, height=_height})<-[r:LINKS]-(p:block{id:$prev_block}) "
                        "RETURN n,r,p",
                        block_id=block_id, version=version, prev_block=prev_block, merkle_root=merkle_root, timestamp=timestamp, 
                        bits=bits, nonce=nonce, n_tx=n_tx)
        return result.single()

    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_output(self, address,tx_id,out_index):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_utxo, address,tx_id,out_index)
            print(result)
            
    def new_tx(self, version, locktime, tx_id, inputs, outputs, segwit):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, tx_id,utxo_local_index_list, outputs)
            print(result)
            
    def new_block(self,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_block,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx)
            print(result)
  
      

In [5]:
db = WalletDB( "neo4j://localhost:7687" , "neo4j" , "wallet" )


OSError: [Errno 0] Error

In [140]:
db.new_address("test_address1",0,False)
db.new_address("test_address2",777,False)
db.new_address("test_address3",22222,False)
db.new_address("test_address4",92747,True)
db.new_address("test_address5",99199191,True)
db.new_address("test_address6",666,True)
db.new_address("test_address7",625,True)

<Record a=<Path start=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586244466132}> end=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'acc_index': 0, 'created': 1586244466132}> size=0>>
<Record a=<Path start=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586244466145}> end=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'acc_index': 777, 'created': 1586244466145}> size=0>>
<Record a=<Path start=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586244466152}> end=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'acc_index': 22222, 'created': 1586244466152}> size=0>>
<Record a=<Path start=<Node id=8 labels={'address'} properties={'address': 

In [141]:
db.new_utxo("test_address1",1000,2)
db.new_utxo("test_address2",1001,0)
db.new_utxo("test_address3",1002,1)
db.new_utxo("test_address4",1003,8)
db.new_utxo("test_address5",1004,5)

<Record p=<Path start=<Node id=6 labels={'utxo'} properties={'transaction_id': 1000, 'out_index': 2, 'local_index': 0, 'address': 'test_address1', 'spent': False}> end=<Node id=14 labels={'address'} properties={'address': 'test_address1', 'type': 'recipient', 'created': 1586244466132, 'acc_index': 0}> size=1>>
<Record p=<Path start=<Node id=7 labels={'utxo'} properties={'transaction_id': 1001, 'out_index': 0, 'local_index': 1, 'address': 'test_address2', 'spent': False}> end=<Node id=15 labels={'address'} properties={'address': 'test_address2', 'type': 'recipient', 'created': 1586244466145, 'acc_index': 777}> size=1>>
<Record p=<Path start=<Node id=9 labels={'utxo'} properties={'transaction_id': 1002, 'out_index': 1, 'local_index': 2, 'address': 'test_address3', 'spent': False}> end=<Node id=0 labels={'address'} properties={'address': 'test_address3', 'type': 'recipient', 'created': 1586244466152, 'acc_index': 22222}> size=1>>
<Record p=<Path start=<Node id=10 labels={'utxo'} propertie

In [142]:
db.new_tx(2222,[0,1], ["ouptut1","output2"])

None


In [143]:
db.update_confirmations(2222,7)

<Record n=<Node id=12 labels={'TxOut'} properties={'outputs': ['ouptut1', 'output2'], 'id': 2222, 'confirmations': 7, 'created': 1586244466250, 'inputs': [0, 1]}>>


In [144]:
up = db.update_utxo(2222)

In [145]:
db.clean_addresses()

None


In [161]:
add = db.look_for_coins()

In [162]:
add

[{'coin.transaction_id': 1002,
  'coin.out_index': 1,
  'coin.address': 'test_address3'},
 {'coin.transaction_id': 1003,
  'coin.out_index': 8,
  'coin.address': 'test_address4'},
 {'coin.transaction_id': 1004,
  'coin.out_index': 5,
  'coin.address': 'test_address5'}]

In [168]:
unused_add = db.get_unused_addresses()
unused_add

[{'unused_address.address': 'test_address6',
  'unused_address.acc_index': 666,
  'unused_address.type': 'change'},
 {'unused_address.address': 'test_address7',
  'unused_address.acc_index': 625,
  'unused_address.type': 'change'}]

In [4]:
driver = GraphDatabase.driver("neo4j://localhost:7687" , auth=("neo4j", "wallet"), encrypted=False)

In [30]:
db.run_as_system()

CypherSyntaxError: Invalid input ':': expected <init> (line 1, column 1 (offset: 0))
":use system RETURN a"
 ^

In [ ]:
import subprocess
import os
def parse_demo(file):
    
    with open(f"blocks_demo/{file}","br") as blockfile:
        c = 0
        try:
            coursor = open("blocks_demo/blk00000.txt")
            c = coursor.readline()
            try:
                c = int(c)
            except:
                if c == "finished":
                    print(c)
                    return True
                else:
                    try: 
                        print("trying to recover file from backup.")
                        coursor = open("blocks_demo/blk00000.txt.bck")
                        c = coursor.readline()
                        try:
                            c = int(c)
                            subprocess.call(f" cp blocks_demo/blk00000.txt.bck blocks_demo/blk00000.txt", shell=True)
                            print("succesfully recovered file from backup. Restored original file.")
                        except:
                            if c == "finished":
                                print(c)
                                subprocess.call(f" cp blocks_demo/blk00000.txt.bck blocks_demo/blk00000.txt",shell=True)
                                print("Finished. Succesfully updated original file.")
                                return True
                            else:
                                print("Corrupted coursor file.")
                                return False
                        
                    except:
                        print("Corrupted coursor file.")
                        return False
                            
            print(c)
            blockfile.read(c)
            coursor.close()

        except:
            coursor  = open("blocks_demo/blk00000.txt","w")
            print("No coursor file")
            coursor.write("\n"+str(c))
            coursor.close()

        #this is the while true in the parse block method:
        while True:
            ccc = blockfile.read(80)
            if ccc == b'': 
                c = "finished"
                coursor  = open("blocks_demo/blk00000.txt","w")
                coursor.write(str(c)+f"\nLast block at {c} byte.")
                coursor.close()
                print(f"Last block at {prev}\nFinished file {file}.")
                return True
            else:c+=80
            coursor  = open("blocks_demo/blk00000.txt","w")
            coursor.write(str(c))
            coursor.close()
            subprocess.call(f" cp blocks_demo/blk00000.txt blocks_demo/blk00000.txt.bck",shell=True)
    
parse_demo("blk00000.dat")        

trying to recover file from backup.
succesfully recovered file from backup. Restored original file.
9011520


In [21]:
with open("blocks_demo/blk00000.dat","br") as blockfile:
    
    for i in range(100):
        print("\n")
        print(blockfile.read(100))
    



b'\xf9\xbe\xb4\xd9\x1d\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00;\xa3\xed\xfdz{\x12\xb2z\xc7,>gv\x8fa\x7f\xc8\x1b\xc3\x88\x8aQ2:\x9f\xb8\xaaK\x1e^J)\xab_I\xff\xff\x00\x1d\x1d\xac+|\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00'


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xffM\x04\xff\xff\x00\x1d\x01\x04EThe Times 03/Jan/2009 Chancellor on brink of second bailout f'


b"or banks\xff\xff\xff\xff\x01\x00\xf2\x05*\x01\x00\x00\x00CA\x04g\x8a\xfd\xb0\xfeUH'\x19g\xf1\xa6q0\xb7\x10\\\xd6\xa8(\xe09\t\xa6yb\xe0\xea\x1fa\xde\xb6I\xf6\xbc?L\xef8\xc4\xf3U\x04\xe5\x1e\xc1\x12\xde\\8M\xf7\xba\x0b\x8dW\x8aLp+k\xf1\x1d_\xac\x00\x00\x00\x00\xf9\xbe\xb4\xd9\xd7\x00\x00"


b'\x00\x01\x00\x00\x00o\xe2\x8c\n\xb6\xf1\xb3r\xc1\xa6\xa2F\xaec\xf7O\x93\x1e\x83e\xe1Z\x08\x9ch\xd6\x19\x00\x00\x00\x00\x00\x98 Q\xfd\x1eK\xa7

In [66]:
from multiprocessing import Pool
from time import sleep
import concurrent.futures

def test_func(file):
    sleep(1)
    print(f".")
    sleep(1)
    print(f"From {file}")
    sleep(1)
    print(f".")
    return
    
    
def main(args):
    n_threads = args[1]
    n = args[0]
    
    file_list = [f"{i:05}" for i in range( n , n + n_threads )]
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads+1) as executor:
        executor.map(test_func, file_list)  
    
def super_main(start_at,n_processes,n_threads):
    batch_list = [start_at+n_threads*i for i in range(n_processes)]
    arg_list = [(n,n_threads) for n in batch_list]
    with Pool(5) as p:
        p.map(main, arg_list)

In [68]:
def manager(total_files, n_processes, n_threads, n_nodes):
    concurrent_files = n_processes*n_threads
    batches = total_files//concurrent_files+1
    batch_list = [concurrent_files*i for i in range(batches)]
    print(f"batch_list {batch_list}")
    for index,batch in enumerate(batch_list):
        node = index%n_nodes
        print(f"node {node} batch {batch}")
        super_main(batch, n_processes, n_threads)
    

manager(100, 4,2,1)

batch_list [0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96]
node 0 batch 0
start at: 0
batch list from supermain: [0, 2, 4, 6]
arg_list from supermain: [(0, 2), (2, 2), (4, 2), (6, 2)]
.
.
.
.
.
.
.
.
From 00004
From 00003
From 00006
From 00000
From 00001
From 00005
From 00007
From 00002
.
.
.
.
.
.
.
.
node 0 batch 8
start at: 8
batch list from supermain: [8, 10, 12, 14]
arg_list from supermain: [(8, 2), (10, 2), (12, 2), (14, 2)]
.
.
.
.
.
.
.
.
From 00014
From 00008
From 00009
From 00015
From 00010
From 00012
From 00011
From 00013
.
.
.
.
.
.
.
.
node 0 batch 16
start at: 16
batch list from supermain: [16, 18, 20, 22]
arg_list from supermain: [(16, 2), (18, 2), (20, 2), (22, 2)]
.
.
.
.
.
.
.
.
From 00021
From 00023
From 00022
From 00017
From 00018
From 00016
From 00020
From 00019
.
.
.
.
.
.
.
.
node 0 batch 24
start at: 24
batch list from supermain: [24, 26, 28, 30]
arg_list from supermain: [(24, 2), (26, 2), (28, 2), (30, 2)]
.
.
.
.
.
.
.
.
From 00028
From 00024
From 00025
Fro

In [57]:
for c in range(8):
    test_func("file")

first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
first from file
second from file
Third from file
